<a href="https://colab.research.google.com/github/santolina/Tensorflow-Dev/blob/master/Image_Style_Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Perceptual Losses for Real-Time Style Transfer and Super-Resolution

In [1]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import glob


In [0]:
# define residual block

def residual_block(input_ts):
  # functional AP
  x = tf.keras.layers.Conv2D(128, (3,3), strides=1, padding='same')(input_ts)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Activation('relu')(x)
  x = tf.keras.layers.Conv2D(128, (3,3), strides=1, padding='same')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  return tf.keras.layers.Add()([x, input_ts])


In [0]:
# define encoder_decoder function

def transfer_network(input_shape=(224, 224, 3)):
  # encoder
  input_ts = tf.keras.layers.Input(shape=input_shape, name='input')

  # normalize input value to [0, 1]
  x = tf.keras.layers.Lambda(lambda a: a/255.)(input_ts)

  x = tf.keras.layers.Conv2D(32, (9, 9), strides=1, padding='same')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Activation('relu')(x)

  x = tf.keras.layers.Conv2D(64, (3, 3), strides=2, padding='same')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Activation('relu')(x)
  
  x = tf.keras.layers.Conv2D(128, (3, 3), strides=2, padding='same')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Activation('relu')(x)

  for _ in range(5):
    x = residual_block(x)

  # decoder
  x = tf.keras.layers.Conv2DTranspose(64, (3,3), strides=2, padding='same')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Activation('relu')(x)

  x = tf.keras.layers.Conv2DTranspose(32, (3,3), strides=2, padding='same')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Activation('relu')(x)

  x = tf.keras.layers.Conv2DTranspose(3, (9,9), strides=1, padding='same')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Activation('tanh')(x)

  # re-normalize output value to [0, 255]
  gen_out = tf.keras.layers.Lambda(lambda a: (a+1)*127.5)(x)

  model_gen = tf.keras.Model(
      inputs = [input_ts],
      outputs = [gen_out]
  )

  return model_gen


In [0]:
input_shape=(224, 224, 3)

model_gen = transfer_network(input_shape=input_shape)

In [6]:
model_gen.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 224, 224, 3)  0           input[0][0]                      
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 224, 224, 32) 7808        lambda[0][0]                     
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 224, 224, 32) 128         conv2d[0][0]                     
______________________________________________________________________________________________

In [7]:
# define loss_network

from tensorflow.keras.applications.vgg16 import VGG16

# call VGG16 model
vgg16 = VGG16()

# freeze weight parameters
for layer in vgg16.layers:
  layer.trainable = False

# preprocessing
def norm_vgg16(x):
  """ RGB->BGR conversion + centorize"""
  return (x[:, :, :, ::-1] - 120) / 255.

# define name for each layer to extract features
style_layer_names = (
    'block1_conv2',
    'block2_conv2',
    'block3_conv3',
    'block4_conv3'
)

content_layer_names = ('block3_conv3', )


# List to store intermidiate outputs
style_outputs_gen = []
contents_outputs_gen = []

input_gen = model_gen.output # input is transfer_network's output

z = tf.keras.layers.Lambda(norm_vgg16)(input_gen)

for layer in vgg16.layers:
  z = layer(z)
  if layer.name in style_layer_names:
    style_outputs_gen.append(z)

  if layer.name in contents_layer_names:
    conent_outputs_gen.append(z)

# define model
model = tf.keras.Model(
    inputs = model_gen.input,
    outputs = style_outputs_gen + contents_outputs_gen
)


553467904/553467096 [==============================] - 20s 0us/step


NameError: ignored